<a href="https://colab.research.google.com/github/Malumatra/AutoGPT1/blob/master/FREE_AUTO_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#USE AI AGENTs, like AUTOGPT or BABYAGI, without paids API. TOTALLY FOR FREE 🎁

## SETUP REPOSITORY 📂


In [4]:
!git clone https://github.com/IntelligenzaArtificiale/Free-AUTO-GPT-with-NO-API.git


fatal: destination path 'Free-AUTO-GPT-with-NO-API' already exists and is not an empty directory.


In [5]:
import pathlib
import re
import subprocess
import typing


def is_in_jupyter_notebook() -> bool:
    """It checks whether a Jupyter notebook is being run"""
    try:
        get_ipython
        return True
    except NameError:
        return False


def is_on_gcolab() -> bool:
    """It checks whether a Jupyter notebook is being run on Google Colab"""
    if not is_in_jupyter_notebook():
        return False

    return "google.colab" in str(get_ipython())


def is_ubuntu_20_04() -> bool:
    import lsb_release
    metadata = lsb_release.get_os_release()

    distro  = metadata["ID"].lower()
    release = metadata["RELEASE"]

    return distro == "ubuntu" and release == "20.04"


def setup_ubuntu_20_04() -> None:
    """It sets up a Ubuntu 20.04 container with `chromium-browser`

    For more information, see
    https://github.com/googlecolab/colabtools/issues/3347#issuecomment-1387453484
    """
    # It adds debian buster
    EOF_debian_buster = """\
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
"""
    !echo "$EOF_debian_buster" > /etc/apt/sources.list.d/debian.list

    # It adds keys
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

    !apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
    !apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
    !apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

    # It adds the debian repo for chromium* packages only
    # Note the double-blank lines between entries
    EOF_chromium_pref = """\
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
"""
    !echo "$EOF_chromium_pref" > /etc/apt/preferences.d/chromium.pref

    # It installs the packages
    !apt-get update
    !apt-get install chromium chromium-driver
    !apt-get install -y xvfb


def setup_requirements() -> None:
    PIP_requirements = " ".join([
        "PyVirtualDisplay", # To run a virtual display
        "undetected-chromedriver==3.2.1",
    ])

    !python3 -m pip install --upgrade $PIP_requirements


def get_py_module_path(module: str) -> typing.Optional[pathlib.Path]:
    """It gets the absolute path of a Python module"""
    r = subprocess.run(
        ["pip", "show", module],
        capture_output=True
    )

    try:
        r.check_returncode()
    except subprocess.CalledProcessError:
        return None

    stdout = r.stdout.decode()

    try:
        RE_abspath = "\nLocation: (?P<abspath>.*)\n"

        matches = re.search(RE_abspath, stdout)
        abspath = matches.group("abspath")
    except AttributeError:
        return None

    dist_packages = pathlib.Path(abspath).resolve()
    return dist_packages / module


def patch_undetected_chromedriver() -> None:
    """It forces `undetected_chromedriver` to run the Chromium webdriver

    For more information, see
    https://github.com/ultrafunkamsterdam/undetected-chromedriver/issues/108#issuecomment-1170269377
    """
    chromedriver_filename = "chromedriver_linux64.zip"

    src_chromedriver_filepath = ROOT / chromedriver_filename
    dst_chromedriver_filepath = pathlib.Path("/tmp") / chromedriver_filename

    !zip -j "$src_chromedriver_filepath" /usr/bin/chromedriver

    PY_module = "undetected_chromedriver"
    module_path = get_py_module_path(PY_module)

    patcher_filepath = module_path / "patcher.py"

    with patcher_filepath.open("rt") as f:
        contents = f.read()

    src = f"'file://{src_chromedriver_filepath}'"
    dst = f"'{dst_chromedriver_filepath}'"

    # It is forced to use the local webdriver
    contents = contents.replace(
        f"return urlretrieve(u)[0]",
        f"return urlretrieve({src}, filename={dst})[0]"
    )

    with patcher_filepath.open("wt") as f:
        f.write(contents)


def setup_container() -> None:
    """It sets up the container which is being run"""
    if is_ubuntu_20_04():
        setup_ubuntu_20_04()

    setup_requirements()
    patch_undetected_chromedriver()


ROOT = pathlib.Path("/content")
anchor = ROOT / "anchor.txt"


assert is_on_gcolab(), "It seems you are not on Google Colab"

# It will set the Google Colab container up only
# after disconnections, not after restarts
if not anchor.exists():
    setup_container()
    anchor.touch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 839.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.2.1-py3-none-any.whl size=46437 sha256=0a4554f7c5a9286d501b76505160efa559748b3c8452d48f956906bf0a82e5b1
  Stored in directory: /root/.cache/pip/wheels/88/42/a0/dca35bf77259b8fc5d73cf5613536145b12b38626d5a7ba81b
Successfully built undetected-chromedriver
  Attempting uninstall: undetected-chromedriver
    Found existing installation: undetected-chromedriver 3.5.5
    Uninstalling undetected-chromedriver-3.5.5:
      Successfully uninstalled undetected-chromedriver-3.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gpt4-openai-api 0.9.0 requires undetected-chromedriver>=3.5.3, but you have undetected-chromedriver 3.2.1 which is incompatible.
	zip warning: name 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -r /content/Free-AUTO-GPT-with-NO-API/requirements.txt
# install dependencies
!pip install PyVirtualDisplay
!pip install langchain.experimental


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

## HOW TO GET TOKEN and Cookies FREE 💡




- **HUGGINGFACE🤗** : Visit this simple [official guide](https://huggingface.co/docs/hub/security-tokens)

- **Now Running also with HuggingCHAT**

- **(OPTIONAL BUT BETTER RESULT) CHATGPT🖥** :
    1. Go to https://chat.openai.com/chat and open the developer tools by `F12`.
    2. Find the `__Secure-next-auth.session-token` cookie in `Application` > `Storage` > `Cookies` > `https://chat.openai.com`.
    3. Copy the value in the `Cookie Value` field.


![image](https://user-images.githubusercontent.com/19218518/206170122-61fbe94f-4b0c-4782-a344-e26ac0d4e2a7.png)

- **(OPTIONAL) Google Bard🖥** :
    1. Go toGo to https://bard.google.com/ and open the developer tools by `F12`.
    2. Find the `__Secure-1PSID` cookie in `Application` > `Storage` > `Cookies`
    3. Copy the value in the `Cookie Value` field.

![Cattura](https://user-images.githubusercontent.com/108482353/236518416-ba0fb89c-080d-4e5e-8514-4ed7ac897b55.PNG)



- **(OPTIONAL) Bing CHAT** :
    1. Check if you have access to [Bing Chat](https://chrome.google.com/webstore/detail/bing-chat-for-all-browser/jofbglonpbndadajbafmmaklbfbkggpo)
    2. Install the cookie editor extension for [Chrome](https://chrome.google.com/webstore/detail/cookie-editor/hlkenndednhfkekhgcdicdfddnkalmdm) or [Firefox](https://addons.mozilla.org/en-US/firefox/addon/cookie-editor/)
    3. Go to `bing.com`
    4. Open the extension for cookie
    5. Click "Export" on the bottom right, then "Export as JSON" (This saves your cookies to clipboard)
    6. Paste your cookies into a file `/content/Free-AUTO-GPT-with-NO-API/cookiesBing.json`


![image](https://user-images.githubusercontent.com/108482353/236259872-faf7946c-5648-4733-8d66-978040eacd85.png)


## 🚀SAVE HERE YOURS TOKENs🚀



In [7]:
import os

# Hugging FACE
os.environ["HUGGINGFACE_TOKEN"] = "COPY-HERE-TOKEN"

# ChatGPT ( NOT WORKING ON GOOGLE COLAB )
os.environ["CHATGPT_TOKEN"] = "COPY-HERE-TOKEN"

# BING chat ( rember to copy yours cookies into : /content/Free-AUTO-GPT-with-NO-API/cookiesBing.json )
os.environ["BINGCHAT_COOKIEPATH"] = "/content/Free-AUTO-GPT-with-NO-API/cookiesBing.json"

# Google Bard
os.environ["BARDCHAT_TOKEN"] = "COPY-HERE-COOKIE"



# ⚠️ DONT RUN **CHATGPT** ISTANCE ON COLAB DOSENT WORK FOR NOW ⚠️
# ⚠️ USE ONLY HUGGING CHAT ⚠️

## RUN Other Agent or Ddevelop your own Agent 👨‍💻 🧑‍💻

In [8]:
# Check for available AGENT
!ls /content/Free-AUTO-GPT-with-NO-API/OtherAgent/



csvAgent.py  customAgent.py  FreeLLM  pythonAgent.py  startup.csv  Tool


In [9]:
# open /content/Free-AUTO-GPT-with-NO-API/OtherAgent/FreeLLM/pyChatGPT.py
#


!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/csvAgent.py

#insert this path for csv : /content/Free-AUTO-GPT-with-NO-API/OtherAgent/startup.csv

Traceback (most recent call last):
  File "/content/Free-AUTO-GPT-with-NO-API/OtherAgent/csvAgent.py", line 4, in <module>
    from langchain.agents import create_csv_agent
  File "/usr/local/lib/python3.10/dist-packages/langchain/agents/__init__.py", line 127, in __getattr__
    raise ImportError(
ImportError: create_csv_agent has been moved to langchain experimental. See https://github.com/langchain-ai/langchain/discussions/11680for more information.
Please update your import statement from: `langchain.agents.create_csv_agent` to `langchain_experimental.agents.create_csv_agent`.


In [10]:
!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/pythonAgent.py

Traceback (most recent call last):
  File "/content/Free-AUTO-GPT-with-NO-API/OtherAgent/pythonAgent.py", line 4, in <module>
    from langchain.agents.agent_toolkits import create_python_agent
  File "<frozen importlib._bootstrap>", line 1075, in _handle_fromlist
  File "/usr/local/lib/python3.10/dist-packages/langchain/agents/agent_toolkits/__init__.py", line 120, in __getattr__
    relative_path = as_import_path(Path(__file__).parent, suffix=name)
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/_api/path.py", line 30, in as_import_path
    path = get_relative_path(file, relative_to=relative_to)
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/_api/path.py", line 18, in get_relative_path
    return str(file.relative_to(relative_to))
  File "/usr/lib/python3.10/pathlib.py", line 818, in relative_to
    raise ValueError("{!r} is not in the subpath of {!r}"
ValueError: '/usr/local/lib/python3.10/dist-packages/langchain/agents/agent_toolkits' is not in the su

In [11]:
!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/customAgent.py

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain/_api/module_import.py", line 69, in import_by_name
    module = importlib.import_module(new_module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'langchain_community'

The above exception was the direct cause of the following exception:

Traceback (most recent call la

## RUN BABYAGI 👶

In [12]:
#Open the /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py and Set "verbose=True" for view the debug

!python /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain/_api/module_import.py", line 69, in import_by_name
    module = importlib.import_module(new_module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'langchain_community'

The above exception was the direct cause of the following exception:

Traceback (most recent call la

## RUN Meta-Prompt 👥

In [13]:
!python /content/Free-AUTO-GPT-with-NO-API/MetaPrompt.py

/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:29: UserWarning: Importing LLMChain from langchain root module is no longer supported. Please use langchain.chains.LLMChain instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:29: UserWarning: Importing PromptTemplate from langchain root module is no longer supported. Please use langchain_core.prompts.PromptTemplate instead.
  warnings.warn(
Traceback (most recent call last):
  File "/content/Free-AUTO-GPT-with-NO-API/MetaPrompt.py", line 8, in <module>
    from FreeLLM import BingChatAPI  # FREE BINGCHAT API
  File "/content/Free-AUTO-GPT-with-NO-API/FreeLLM/BingChatAPI.py", line 1, in <module>
    from EdgeGPT import Chatbot, ConversationStyle
ImportError: cannot import name 'Chatbot' from 'EdgeGPT' (/usr/local/lib/python3.10/dist-packages/EdgeGPT/__init__.py)


## RUN AUTOGPT 🤖

In [14]:
#Open the /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py and modify or add new tool
"""
#ACTUAL TOOL
tools = [
    web_search, #DuckDuckGoSearchRun()
    WriteFileTool(root_dir="/data"),
    ReadFileTool(root_dir="/data"),
    process_csv,
    query_website_tool, playwright()
    # HumanInputRun(), # Activate if you want the permit asking for help from the human
]
"""
!python /content/Free-AUTO-GPT-with-NO-API/AUTOGPT.py

Traceback (most recent call last):
  File "/content/Free-AUTO-GPT-with-NO-API/AUTOGPT.py", line 11, in <module>
    from langchain.experimental.autonomous_agents.autogpt.agent import AutoGPT
ModuleNotFoundError: No module named 'langchain.experimental'


## RUN CAMEL with Streamlit.io 🐪

---


In [15]:
# SETUP COLAB for run Streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.427s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run /content/Free-AUTO-GPT-with-NO-API/Camel.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 2.154s
your url is: https://tired-cases-wear.loca.lt
